In [11]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from src.data.transforms import (
    load_data,
	impute_position_change_type,
	fill_percent_fields,
	fill_avg_price,
	fill_current_shares,
    aggregate_investor_holdings,
    aggregate_company_holdings,
    engineer_investor_features,
    engineer_company_features
)

In [12]:
orgs_csv_path = '../data/raw/orgs.csv'
holdings_csv_path = '../data/raw/holdings.csv'

df_orgs, df_holdings = load_data(orgs_csv_path, holdings_csv_path)

# Display the first few rows of each DataFrame
print("Orgs Data:")
display(df_orgs.head())

print("Holdings Data:")
display(df_holdings.head())

Orgs Data:


,email_extension,org_id,cik,ticker,stock_id,filer_id,bio,org_type
0,solelpartners.com,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,1790604.0,NaN,NaN,463912.0,Solel Partners is a hedge fund manager based i...,INSTITUTIONAL_INVESTOR
1,alvariuminvestments.com,3b92ee8f-9761-484b-ae78-2f3d30eb19dd,NaN,ALTI,6941.0,NaN,"AlTi Global, Inc. is a publicly traded interna...",PUBLIC_COMPANY
2,aprio.com,16315875-6b1e-4adf-90c1-c6f16892bc16,1803519.0,NaN,NaN,463917.0,Aprio is a premier business advisory and accou...,INSTITUTIONAL_INVESTOR
3,ctcmycfo.com,259600df-96b9-4117-ad22-2fa07dea2528,1673962.0,NaN,NaN,336130.0,BMO Family Office is an integrated wealth mana...,INSTITUTIONAL_INVESTOR
4,boeing.com,c2696c82-1847-4cef-9665-6f893006579e,NaN,BA,210.0,NaN,Boeing Company (The) is a leading global aeros...,PUBLIC_COMPANY


Holdings Data:


,email_extension,id,org_id,filer_id,filer_name,stock_id,stock_name,stock_ticker,security_type,shares_change,...,industry,percent_ownership,filer_street_address,filer_city,filer_state,filer_zip_code,avg_price,percent_change,quarter_id_owned,quarter_end_price
0,solelpartners.com,b5e6446d-a8ec-470d-b234-e677f28e3d4c,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,463912.0,SOLEL PARTNERS LP,221.0,CVS Health Corp,CVS,SH,625500.0,...,RETAIL - FOOD & DRUG,0.0897,NaN,NaN,NaN,NaN,62.0177,124.26,NaN,NaN
1,paradigm.com,bf8029ca-24b2-48c0-b064-6d639fa32bf3,a60fe97c-3839-463d-8937-68e8e0cee430,2158.0,PARADIGM CAPITAL MANAGEMENT IN,6948.0,American Woodmark Corp,AMWD,SH,301189.0,...,CONSTRUCTION & ENGINEERING,2.0007,NaN,NaN,NaN,NaN,90.8964,NaN,NaN,NaN
2,ruffer.co.uk,15d99797-69cc-4bf8-873d-04ae9a470d61,a9897469-0b3b-4de3-94b2-fc3d84ef665f,2508.0,RUFFER LLP,6021.0,First Solar Inc,FSLR,SH,30480.0,...,ELECTRICAL EQUIPMENT,0.0285,NaN,NaN,NaN,NaN,207.1650,NaN,NaN,NaN
3,solelpartners.com,a861cb00-eeab-40a0-82ca-3959271e5a2f,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,463912.0,SOLEL PARTNERS LP,211769.0,Grocery Outlet Holding Corp,GO,SH,813627.0,...,FOOD PRODUCTS,2.4902,NaN,NaN,NaN,NaN,22.8637,50.02,NaN,NaN
4,solelpartners.com,145f8c19-c53b-481e-8941-face43a8d48c,75b97ec2-ddb1-4a7a-b97a-359d9658e97d,463912.0,SOLEL PARTNERS LP,11343.0,Cigna Holding Co,CI,SH,-2900.0,...,INSURANCE,0.0387,NaN,NaN,NaN,NaN,226.5949,-2.61,NaN,NaN


In [13]:
print(f"orgs rows {len(df_orgs)} and holdings rows {len(df_holdings)}")

orgs rows 3630 and holdings rows 174378


In [14]:
df_holdings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174378 entries, 0 to 174377
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   email_extension                174378 non-null  object        
 1   id                             173949 non-null  object        
 2   org_id                         173949 non-null  object        
 3   filer_id                       173949 non-null  float64       
 4   filer_name                     172823 non-null  object        
 5   stock_id                       173949 non-null  float64       
 6   stock_name                     173949 non-null  object        
 7   stock_ticker                   173927 non-null  object        
 8   security_type                  173949 non-null  object        
 9   shares_change                  173949 non-null  float64       
 10  position_change_type           153606 non-null  object        
 11  

In [15]:
import pandas as pd


df_holdings=df_holdings.drop(columns=['source', 'filer_street_address', 'filer_city','filer_zip_code','filer_zip_code', 'filer_state', 'quarter_id_owned', 'quarter_end_price']) # no data
df_holdings = df_holdings.dropna(subset=['stock_name','stock_ticker']) # have a lot of accompanying missing info; in future spot check how to impute
df_holdings[['current_mv', 'previous_mv', 'previous_percent_of_portfolio', 'current_percent_of_portfolio']]=df_holdings[['current_mv', 'previous_mv', 'previous_percent_of_portfolio', 'current_percent_of_portfolio']].fillna(0.0)
df_holdings['current_ranking'] = df_holdings['current_ranking'].fillna(999999)
df_holdings['previous_ranking'] = df_holdings['previous_ranking'].fillna(999999)
df_holdings['filer_id'] = df_holdings['filer_id'].fillna(0)
df_holdings['filer_name'] = df_holdings['filer_name'].fillna('unassigned filer')
df_holdings = df_holdings[~df_holdings['stock_ticker'].isin(['0', '660', '0R87N:MEXI', '11G.OLD:FWB2', '9688.HK'])]
df_holdings['previous_shares'] = df_holdings['previous_shares'].fillna(0)
df_holdings['current_shares'] = df_holdings.apply(fill_current_shares, axis=1)
# Now drop rows where current_shares is still NaN
df_holdings = df_holdings.dropna(subset=['current_shares'])
df_holdings['position_change_type'] = df_holdings.apply(impute_position_change_type, axis=1)
df_holdings[['sector', 'industry']] = df_holdings[['sector', 'industry']].fillna('Unknown')
df_holdings = df_holdings.apply(fill_percent_fields, axis=1)
sector_medians = df_holdings.groupby('sector')['avg_price'].median()
df_holdings['avg_price'] = df_holdings.apply(lambda row: fill_avg_price(row, sector_medians), axis=1)
df_holdings.info()


<class 'pandas.core.frame.DataFrame'>
Index: 173922 entries, 0 to 173948
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   email_extension                173922 non-null  object        
 1   id                             173922 non-null  object        
 2   org_id                         173922 non-null  object        
 3   filer_id                       173922 non-null  float64       
 4   filer_name                     173922 non-null  object        
 5   stock_id                       173922 non-null  float64       
 6   stock_name                     173922 non-null  object        
 7   stock_ticker                   173922 non-null  object        
 8   security_type                  173922 non-null  object        
 9   shares_change                  173922 non-null  float64       
 10  position_change_type           173922 non-null  object        
 11  curre

In [16]:
# Aggregate investor holdings data
investor_agg = aggregate_investor_holdings(df_holdings)
print("Aggregated Investor Holdings:")
display(investor_agg.head())
# Aggregate company holdings data
company_agg = aggregate_company_holdings(df_holdings)
print("Aggregated Company Holdings:")
display(company_agg.head())

Aggregated Investor Holdings:


,org_id,shares_change_sum,shares_change_mean,current_mv_sum,current_mv_mean,previous_mv_sum,previous_mv_mean,current_percent_of_portfolio_mean,percent_ownership_mean,source_date_<lambda>
0,00015e10-c903-4ce7-87d8-47828f16b640,496768.0,12419.200000,3.070285e+08,7.675712e+06,2.553791e+08,6.384478e+06,2.391220,0.030453,0
1,000598f6-8db0-4c94-87b8-137f04ff95c1,269098.0,2718.161616,1.277503e+08,1.290407e+06,1.257624e+08,1.270327e+06,1.030393,0.369709,0
2,001dbc3c-0151-49ff-a38e-bbab8c75d4e1,-2461366.0,-26753.978261,1.181313e+10,1.284036e+08,1.076643e+10,1.170264e+08,0.151907,0.857720,0
3,001e71f1-cc54-481b-88d1-1359ee62fb71,39171.0,39171.000000,2.188362e+08,2.188362e+08,2.060970e+08,2.060970e+08,0.299200,0.092200,0
4,00286966-c055-4807-a10a-5b6adaca7b2a,-2726816.0,-64924.190476,5.196324e+08,1.237220e+07,5.585555e+08,1.329894e+07,0.603378,0.664552,0


Aggregated Company Holdings:


,stock_id,filer_id_count,current_mv_sum,current_mv_mean,current_percent_of_portfolio_mean,percent_ownership_mean
0,1.0,164,4.508334e+10,2.748984e+08,0.794100,0.448955
1,2.0,4,0.000000e+00,0.000000e+00,0.000000,0.000000
2,4.0,126,1.319901e+10,1.047541e+08,0.268986,0.689667
3,5.0,31,4.535674e+08,1.463121e+07,1.055808,0.172158
4,6.0,1,0.000000e+00,0.000000e+00,0.000000,0.000000
